In [2]:
import editdistance
# huggingface's transformers library
from transformers import BertModel, BertTokenizer

/home/sinha/miniconda3/envs/gml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from itertools import product

In [4]:
from nltk.util import ngrams

In [5]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# from evaluate import load

# bertscore = load("bertscore")

In [7]:
def jaccard_similarity(str1, str2, n):
    str1_bigrams = list(ngrams(str1, n))
    str2_bigrams = list(ngrams(str2, n))

    intersection = len(list(set(str1_bigrams).intersection(set(str2_bigrams))))
    union = (len(set(str1_bigrams)) + len(set(str2_bigrams))) - intersection

    return float(intersection) / union

In [8]:
ORIGINAL_KEYS_LIST = [
"orientation",
"paper_size",
"Copies",
"Color Print",
"adjust_font_size", 
"Duplex Printing",
"print_quality",
"font_size",
"column_widths",
"horizontal_alignment",
"text_wrap",
"hide_empty_columns",
"hide_long_columns",
"vertical_alignment",
"increase_header_font",
"show_header_in_all_page"
]
# TO LOWER CASE
NORMALIZED_KEYS_LIST = [ key.lower() for key in ORIGINAL_KEYS_LIST]

# normalized to original map 
NORMALIZED_TO_ORIGINAL_MAP = {NORMALIZED_KEYS_LIST[i]: ORIGINAL_KEYS_LIST[i] for i in range(len(ORIGINAL_KEYS_LIST))}

In [9]:
input_dict = {
    'Color': 'No',
    'Copy': 1,
    'Duplex': 'No',
    'column_widths': True,
    'font size': 'Medium',
    'adjust font size': 'No',
    'hide_empty_columns': False,
    'hide_long_columns': True,
    'horizontal_alignment': True,
    'increase_header_font': True,
    'orientation': 'landscape',
    'paper size': 'A4',
    'show_header_in_all_page': True,
    'vertical_alignment': False
}

In [10]:
# function to find the closest match of the keys using jaccard similarity
def find_closest_match(input_dict, keys_list):
    match = {}
    for key in input_dict.keys():
        key = key.lower()
        max_sim = 0
        for k in keys_list:
            sim = jaccard_similarity(key, k, 3)
            if sim > max_sim:
                max_sim = sim
                match[key] = k

        # map to original key
        match[key] = NORMALIZED_TO_ORIGINAL_MAP[match[key]]
    return match 
    

In [11]:
match = find_closest_match(input_dict, NORMALIZED_KEYS_LIST)

In [12]:
match

{'color': 'Color Print',
 'copy': 'Copies',
 'duplex': 'Duplex Printing',
 'column_widths': 'column_widths',
 'font size': 'font_size',
 'adjust font size': 'adjust_font_size',
 'hide_empty_columns': 'hide_empty_columns',
 'hide_long_columns': 'hide_long_columns',
 'horizontal_alignment': 'horizontal_alignment',
 'increase_header_font': 'increase_header_font',
 'orientation': 'orientation',
 'paper size': 'paper_size',
 'show_header_in_all_page': 'show_header_in_all_page',
 'vertical_alignment': 'vertical_alignment'}